<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [1]:
### YOUR CODE STARTS HERE
import pandas as pd
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel # I know all these arent needed.
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-01-29 23:43:34--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.03s   

2020-01-29 23:43:34 (606 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [3]:
# Making house votes into dataframe
column_headers = ['party', 'handicapped-infants', 'water-project',
                  'budget', 'physician-fee-freeze', 'el-salvador-aid',
                  'religious-groups', 'anti-satellite-ban',
                  'aid-to-contras', 'mx-missle', 'immigration',
                  'synfuels', 'education', 'right-to-sue-', 'crime', 'duty-free',
                  'south-africa']
df = pd.read_csv('house-votes-84.data',
                 header=None,
                 names=column_headers,  # <-- This makes my column headers.
                 na_values='?')   # <-- any missing values or ? will become NaN.

df.shape            

(435, 17)

In [52]:
# Cleaning data by changing yes and no to 1 and 0 respectively for ttest.
df = df.replace({'y':1, 'n': 0})
df.head()  # Making sure this change took effect.

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missle,immigration,synfuels,education,right-to-sue-,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
# Create subsets of data for republicans and democrats.
rep = df[df['party']=='republican']
dem = df[df['party']=='democrat']

In [11]:
# Examining budget bill to compare rep and dem votes manually.
# Rep votes actual numbers
rep['budget'].value_counts()

0.0    142
1.0     22
Name: budget, dtype: int64

In [59]:
# Finding average of rep votes for budget.
print('Percentage of Republicans that voted yes: ''{:.2f}%'.format(rep['budget'].mean()*100))

Percentage of Republicans that voted yes: 13.41%


In [12]:
# Dem votes actual numbers
dem['budget'].value_counts()

1.0    231
0.0     29
Name: budget, dtype: int64

In [60]:
# Finding average of dem votes for budget.
print('Percentage of Democrats that voted yes: ''{:.2f}%'.format(dem['budget'].mean()*100))

Percentage of Democrats that voted yes: 88.85%


In [71]:
# By doing the long way, I see that dems voted yes more than reps did on this.
# Now to run the ttest, while omitting the nans
ttest_ind(rep['budget'], dem['budget'], nan_policy='omit').pvalue

2.0703402795404463e-77

In [72]:
pvalue < 0.01

True

In [32]:
# Now to find a bill that reps support more than dems. Attempting fee freeze.
# Rep votes actual numbers
rep['physician-fee-freeze'].value_counts()

1.0    163
0.0      2
Name: physician-fee-freeze, dtype: int64

In [61]:
# Finding average of rep votes for physician fee freeze.
print('Percentage of Republicans that voted yes: ''{:.2f}%'.format(rep['physician-fee-freeze'].mean()*100))

Percentage of Republicans that voted yes: 98.79%


In [33]:
# Dem votes actual numbers.
dem['physician-fee-freeze'].value_counts()

0.0    245
1.0     14
Name: physician-fee-freeze, dtype: int64

In [62]:
# Finding average of dem votes for physician fee freeze.
print('Percentage of Democrats that voted yes: ''{:.2f}%'.format(dem['physician-fee-freeze'].mean()*100))

Percentage of Democrats that voted yes: 5.41%


In [67]:
# Doing it the long way again, I see that reps voted yes more than dems, now 
# to check the pvalue.
ttest_ind(rep['physician-fee-freeze'], dem['physician-fee-freeze'], nan_policy='omit').pvalue

1.994262314074344e-177

In [68]:
pvalue < 0.01

True

In [66]:
# Per my attempt, I see the pvalue is less than 0.01 and shows reps voted yes
# more than dems did on the physician fee freeze bill.
#
# Now to attempt to find a bill where p is > 0.1  I am going to try using mx-missle.
ttest_ind(rep['mx-missle'], dem['mx-missle'], nan_policy='omit').pvalue

5.03079265310811e-47

In [69]:
pvalue > 0.1

False

In [65]:
# Mx-missle did not work, now I will try water-project.
ttest_ind(rep['water-project'], dem['water-project'], nan_policy='omit').pvalue

0.9291556823993485

In [70]:
pvalue > 0.1

False

In [0]:
# I can visually see that the pvalue is 0.929...  But when I check pvalue greater than
# 0.1, it says false.